In [307]:
import io
import requests
from bs4 import BeautifulSoup
import pandas as pd
from statsbombpy import sb
from django.utils.safestring import mark_safe
import time



In [234]:
data_link_file = "./links/data_by_season_link.txt"

standard_data_link_file = "./links/links_for_data/standard_stat_by_season.txt"
defense_data_link_file = "./links/links_for_data/defensive_stat_by_season.txt"
keepers_data_link_file = "./links/links_for_data/keepers_stat_by_season.txt"
schedule_data_link_file = "./links/links_for_data/schedules_stat_by_season.txt"

In [310]:

# Définissez l'URL de la page d'accueil de la Ligue des champions
website = "https://fbref.com"
url = "https://fbref.com/en/comps/8/history/Champions-League-Seasons"

# Effectuez une requête HTTP pour récupérer la page d'accueil
response = requests.get(url)

def check_access_url(response):
    return response.status_code == 200

def get_source_code(url):
    result =[]
    response = requests.get(url)
    if response.status_code == 429:
        # Attendre le temps indiqué dans l'en-tête Retry-After
        print("prends une pause et reviens dans",response.headers["Retry-After"], "secondes")
        time.sleep(int(response.headers["Retry-After"]))
        result = get_source_code(url)
    
    if check_access_url(response):
        result = BeautifulSoup(response.content, "html.parser")
        return result

def make_comments_code_visible(soup):
    soup = str(soup)
    soup = soup.replace("<!--", "")
    soup = soup.replace("-->", "")
    soup = mark_safe(soup)
    soup = BeautifulSoup(soup, "html.parser")
    result = soup
    return result

def get_table_saisons_link(url):
    response = requests.get(url)
    if check_access_url(response):
        # Créez un objet BeautifulSoup pour représenter la page d'accueil
        soup = BeautifulSoup(response.content, "html.parser")
        # Trouvez la table des campagnes
        table_saisons_ldc = soup.find("table",{"id":"seasons"})
        body_table_saisons_ldc = table_saisons_ldc.find("tbody")
        ths = body_table_saisons_ldc.find_all("th")
        href_values = list()
        for th in ths:
            href = th.find("a")["href"]
            href_values.append(href)
        links = ths.find_all("a")
        href_values = [link["href"] for link in links]

        for tr in body_table_saisons_ldc.find_all("tr"):
            # Extraire les données de chaque tr
            year = tr.find("th")
            cells = tr.find_all("td")
            data_by_season = []
            for cell in cells[:3]:
                data_by_season.append(cell.text)
            
        return href_values
    return 0

def get_table_saisons_ldc(url):
    # Effectuez une requête HTTP pour récupérer la page d'accueil
    response = requests.get(url)
    data =[]
    if check_access_url(response):
        # Créez un objet BeautifulSoup pour représenter la page d'accueil
        soup = BeautifulSoup(response.content, "html.parser")

        # Trouvez la table des campagnes
        table_saisons_ldc = soup.find("table",{"id":"seasons"})
        #print(table_saisons_ldc)

        theads = table_saisons_ldc.find("thead")
        theaders = theads.find_all("th")
        headers = []
        for h in theaders[:5] :
            headers.append(h.text)

        data.append(headers)
        body_table_saisons_ldc = table_saisons_ldc.find("tbody")
        
        for tr in body_table_saisons_ldc.find_all("tr"):
            # Extraire les données de chaque tr
            year = tr.find("th")
            cells = tr.find_all("td")
            data_by_season = []
            data_by_season.append(year.text)
            for cell in cells[:4]:
                data_by_season.append(cell.text)
            data.append(data_by_season)

    return data

def get_url_for_a_season(url, season):
    return url+season

def save_link_in_txt_file(list_data, filename):
  """
  Sauvegarde une liste de données dans un fichier TXT.

  Args:
    list_data: La liste de données à sauvegarder.
    filename: Le nom du fichier TXT.
  """

  with io.open(filename, "w", encoding="utf-8") as f:
    for data in list_data:
      f.write(data + "\n")


# Vérifiez le code de statut de la réponse HTTP    
#filename_for_data_by_season = "data_by_season_link.txt"
#all_links = get_table_saisons_link(url)
#save_link_in_txt_file(all_links, filename_for_data_by_season)



***Récupération des liens menant aux stats d'une édition de ldc à partir du fichier texte contenant les url pour chaque éditions***

In [133]:
def read_data(file):
    """
    Lit les données d'un fichier txt et les renvoie dans un tableau.

    Args:
    fichier: Le chemin du fichier txt à lire.

    Returns:
    Un tableau contenant les données du fichier.
    """
    # Ouvre le fichier en lecture seule
    with open(file, "r") as f:
        content = f.read()
        content = content.split("\n")

    return content


link_data = read_data(data_link_file)
filename_for_standard_stat_by_season = "./links/links_for_data/standard_stat_by_season.txt"
link_standard_stat_by_season = []
idx = 0
for l in link_data :
    l_split = l.split("/")
    #print(l_split)
    if idx>0:
        new_link_standard_list = l_split[1:5]
        new_link_standard_list.append("stats")
        new_link_standard_list.append(l_split[-1])
        new_link = ""
        for nl in new_link_standard_list:
            new_link+=("/"+nl)
        #print(new_link)
        
        link_standard_stat_by_season.append(new_link)
        #print(l_split[-1])
    idx+=1
    
    #save_link_in_txt_file(link_standard_stat_by_season, filename_for_standard_stat_by_season)
    


In [235]:

links_data = dict()
links_data["standard"] = read_data(standard_data_link_file)
links_data["defense"] = read_data(defense_data_link_file)
links_data["keepers"] = read_data(keepers_data_link_file)
links_data["schedule"] = read_data(schedule_data_link_file)

completes_url = dict()
completes_url["standard"] = []
completes_url["defense"] = []
completes_url["keepers"] = []
completes_url["schedule"] = []
def get_complete_urls(links_data, completes_url) :
    for key in links_data.keys():
        for link in links_data[key]:
            if link != "/"+key+"/" and link != "":
                completes_url[key].append(website+link)
    return completes_url

urls = get_complete_urls(links_data=links_data, completes_url=completes_url)
#for urls in completes_url:
#    print(urls)

In [351]:


def get_squads_and_players_stat(key_stats, url_set, id_table_squad_stats, id_table_players_stats):
    stats_standard_data = dict()
    stats_standard_data["squads"] = []
    stats_standard_data["players"] = []
    headers_stats_squads_retrieved = False
    headers_stats_players_retrieved = False
    for url_courant in url_set[key_stats]:
        annee_edition = url_courant.split("/")[6]
        #print(url_courant)
        #if "2023" in url_courant :
        # Créez un objet BeautifulSoup pour représenter la page d'accueil
        soupStats = get_source_code(url_courant)
        if soupStats != []:
            # Trouvez la table des campagnes
            url_code_with_comments = make_comments_code_visible(soupStats)

            table_standard_stat_by_squad = url_code_with_comments.find("table",{"id":id_table_squad_stats})
            table_players_stats_contents = url_code_with_comments.find("table", {"id":id_table_players_stats})
            
            if table_standard_stat_by_squad != None:
                if headers_stats_squads_retrieved == False :
                    headers_stats_squads_retrieved = True
                    theaders_standard_stats_by_squad = table_standard_stat_by_squad.find("thead")
                    headers_standard_stats_by_squad = theaders_standard_stats_by_squad.find_all("tr")
                    headers_squad_standard = []
                    for idx in range(len(headers_standard_stats_by_squad)):
                        if idx == 1:
                            th_headers_standard_stats_by_squad = headers_standard_stats_by_squad[idx].find_all("th")
                            for th in th_headers_standard_stats_by_squad :
                                headers_squad_standard.append(th.text)
                            headers_squad_standard.append("Année édition")
                    stats_standard_data["squads"].append(headers_squad_standard)


                body_table_standard_stat_by_squad = table_standard_stat_by_squad.find("tbody")
                tr_headers_standard_by_squad = body_table_standard_stat_by_squad.find_all("tr")
                for tr in tr_headers_standard_by_squad:
                    row = []
                    a = tr.find("a")
                    row.append(a.text)
                    for td in tr.find_all("td"):
                        row.append(td.text)
                    row.append(annee_edition)
                    stats_standard_data["squads"].append(row)


            if table_players_stats_contents != None :
                theaders_standard_stats_by_players = table_players_stats_contents.find("thead")
                headers_standard_stats_by_players = theaders_standard_stats_by_players.find_all("tr")
                headers_players_standard = []
                for idx in range(len(headers_standard_stats_by_players)):
                    if idx == 1:
                        th_headers_standard_stats_by_players = headers_standard_stats_by_players[idx].find_all("th")
                        for th in th_headers_standard_stats_by_players :
                            headers_players_standard.append(th.text) 
                        headers_players_standard.append("Année édition")
                stats_standard_data["players"].append(headers_players_standard)
                
                body_table_standard_stat_by_players = table_players_stats_contents.find("tbody")
                tr_headers_standard_by_players = body_table_standard_stat_by_players.find_all("tr")
                for tr in tr_headers_standard_by_players:
                    row = []
                    th_row = tr.find("th")
                    a = tr.find("a")
                    if th_row != None:
                        row.append(th_row.text)
                    if a != None :
                        #print(a.text)
                        row.append(a.text)
                    for td in tr.find_all("td"):
                        atd = td.find("a")
                        if atd != None:
                            row.append(atd.text)
                        else:
                            row.append(td.text)
                    row.append(annee_edition)        
                    stats_standard_data["players"].append(row)
    return stats_standard_data     



def save_data_frame_to_csv(data_to_save, file_path):
    data_frame = pd.DataFrame(data=data_to_save[1:])
    #print(data_frame)
    data_frame.to_csv(file_path, index=False, header=data_to_save[0])



In [393]:


def get_players_standard_stat(key_stats, url_set, id_table_players_stats):
    stats_standard_data = dict()
    stats_standard_player_data = []
    headers_stats_squads_retrieved = False
    headers_stats_players_retrieved = False
    for url_courant in url_set[key_stats]:
        annee_edition = url_courant.split("/")[6]
        #print(url_courant)
        #if "2023" in url_courant :
        # Créez un objet BeautifulSoup pour représenter la page d'accueil
        soupStats = get_source_code(url_courant)
        if soupStats != []:
            # Trouvez la table des campagnes
            url_code_with_comments = make_comments_code_visible(soupStats)

            table_players_stats_contents = url_code_with_comments.find("table", {"id":id_table_players_stats})

            if table_players_stats_contents != None :
                theaders_standard_stats_by_players = table_players_stats_contents.find("thead")
                headers_standard_stats_by_players = theaders_standard_stats_by_players.find_all("tr")
                headers_players_standard = []
                for idx in range(len(headers_standard_stats_by_players)):
                    if idx == 1:
                        th_headers_standard_stats_by_players = headers_standard_stats_by_players[idx].find_all("th")
                        for th in th_headers_standard_stats_by_players[:-1] :
                            headers_players_standard.append(th.text) 
                        headers_players_standard.append("Année édition")
                stats_standard_player_data.append(headers_players_standard)
                
                body_table_standard_stat_by_players = table_players_stats_contents.find("tbody")
                tr_headers_standard_by_players = body_table_standard_stat_by_players.find_all("tr")
                for tr in tr_headers_standard_by_players :
                    row = []
                    th_row = tr.find("th")
                    a = tr.find("a")
                    if th_row != None:
                        row.append(th_row.text)
                    if a != None :
                        #print(a.text)
                        row.append(a.text)
                    for td in tr.find_all("td")[:-1]:
                        atd = td.find("a")
                        if atd != None:
                            row.append(atd.text)
                        else:
                            row.append(td.text)
                    row.append(annee_edition)        
                    stats_standard_player_data.append(row)
    return stats_standard_player_data     


In [411]:
#all_standard_stats = get_squads_and_players_stat("standard", urls, "stats_squads_standard_for", "stats_standard")
#all_player_stats = get_players_standard_stat("standard", urls, "stats_standard")
all_player_keeper_stats = get_players_standard_stat("keepers", urls, "stats_keeper")
all_player_defense_stats = get_players_standard_stat("defense", urls, "stats_defense")

prends une pause et reviens dans 3600 secondes
prends une pause et reviens dans 0 secondes


In [409]:
all_player_stats = get_players_standard_stat("standard", urls, "stats_standard")
stats_players_by_year = dict()
#print(all_player_stats[0])
for r in all_player_stats[1:] :
    if r[-1] not in stats_players_by_year.keys():
        stats_players_by_year[r[-1]] = list()
    row = []
    for cell in range(len(r)):
        if cell != 2:
            row.append(r[cell])
    stats_players_by_year[r[-1]].append(row)
    #print(row)

standard_stats_player = list()
standard_stats_player.append(all_player_stats[0])
for d in stats_players_by_year.keys() :
    for i in stats_players_by_year[d]:
        if len(i) > 2:
            #print(i)
            standard_stats_player.append(i)
        #print(i)

#c = 0
"""for p in range(len(standard_stats_player)-1):

    for i in range(len(standard_stats_player[p])):
            print(standard_stats_player[0][i], " : ",standard_stats_player[p+1][i])
        
    print(p)"""

def save_data_player_frame_to_csv(data_to_save, file_path):
    data_frame = pd.DataFrame(data=data_to_save[1:])
    #print(data_to_save[1])
    data_frame.to_csv(file_path, index=False, header=data_to_save[0])

file_path_player_stat_squads = "datas_project/player_stats/standard_player_stats.csv"
save_data_player_frame_to_csv(standard_stats_player, file_path_player_stat_squads)

In [421]:
stats_keeper_players_by_year = dict()
#print(all_player_stats[0])
for r in all_player_keeper_stats[1:] :
    if r[-1] not in stats_keeper_players_by_year.keys():
        stats_keeper_players_by_year[r[-1]] = list()
    row = []
    for cell in range(len(r)):
        if cell != 2:
            row.append(r[cell])
    stats_keeper_players_by_year[r[-1]].append(row)
    #print(row)

keeper_stats_player = list()
keeper_stats_player.append(all_player_keeper_stats[0])
#print(all_player_keeper_stats[0])
for d in stats_keeper_players_by_year.keys() :
    for i in stats_keeper_players_by_year[d]:
        if len(i) > 2:
            #print(i)
            keeper_stats_player.append(i)
        #print(i)
file_path_player_stat_keeper = "datas_project/player_stats/keeper_player_stats.csv"
save_data_player_frame_to_csv(keeper_stats_player, file_path_player_stat_keeper)

In [422]:
stats_players_defense_by_year = dict()
#print(all_player_stats[0])
for r in all_player_defense_stats[1:] :
    if r[-1] not in stats_players_defense_by_year.keys():
        stats_players_defense_by_year[r[-1]] = list()
    row = []
    for cell in range(len(r)):
        if cell != 2:
            row.append(r[cell])
    stats_players_defense_by_year[r[-1]].append(row)
    #print(row)

stats_defense_player = list()
stats_defense_player.append(all_player_defense_stats[0])
for d in stats_players_defense_by_year.keys() :
    for i in stats_players_defense_by_year[d]:
        if len(i) > 2:
            #print(i)
            stats_defense_player.append(i)
        #print(i)
file_path_player_stat_defense = "datas_project/player_stats/defense_player_stats.csv"
save_data_player_frame_to_csv(stats_defense_player, file_path_player_stat_defense)

In [344]:
all_defense_stats = get_squads_and_players_stat("defense", urls, "stats_squads_defense_for", "stats_defense")
"""for url in urls["defense"]:
    checking_url = check_access_url(requests.get(url))
    if checking_url != True:
        print(url)"""

'for url in urls["defense"]:\n    checking_url = check_access_url(requests.get(url))\n    if checking_url != True:\n        print(url)'

In [373]:
file_path_defense_stat_squads = "datas_project/defense_squads_stats.csv"
save_data_frame_to_csv(all_defense_stats["squads"], file_path_defense_stat_squads)

In [355]:
all_keepers_stats = get_squads_and_players_stat("keepers", urls, "stats_squads_keeper_for", "stats_keeper")
"""for url in urls["keepers"]:
    checking_url = check_access_url(requests.get(url))
    if checking_url != True:
        print(url)"""

prends une pause et reviens dans 3600 secondes


'for url in urls["keepers"]:\n    checking_url = check_access_url(requests.get(url))\n    if checking_url != True:\n        print(url)'

In [365]:
#file_path_keeper_stat_squads = "datas_project/keeper_squads_stats.csv"
#save_data_frame_to_csv(all_keepers_stats["squads"], file_path_keeper_stat_squads)
#for k in all_keepers_stats["players"]:
    #for kk in all_keepers_stats[k]:
    #print(k)


In [372]:



def get_schedules_stat(key_stats, url_set, id_table_squad_stats):
    stats_schedules_data = dict()
    stats_schedules_data["matchs"] = []

    headers_stats_squads_retrieved = False
    for url_courant in url_set[key_stats]:
        annee_edition = url_courant.split("/")[6]
        #print(url_courant)
        #if "2023" in url_courant :
        # Créez un objet BeautifulSoup pour représenter la page d'accueil
        soupStats = get_source_code(url_courant)
        if soupStats != []:
            # Trouvez la table des campagnes
            url_code_with_comments = make_comments_code_visible(soupStats)

            table_standard_stat_by_squad = url_code_with_comments.find("table",{"id":id_table_squad_stats})
            
            if table_standard_stat_by_squad != None:
                if headers_stats_squads_retrieved == False :
                    headers_stats_squads_retrieved = True
                    theaders_standard_stats_by_squad = table_standard_stat_by_squad.find("thead")
                    headers_standard_stats_by_squad = theaders_standard_stats_by_squad.find("tr")
                    headers_squad_standard = []
                    for th_header in headers_standard_stats_by_squad.find_all("th"):
                        headers_squad_standard.append(th_header.text)
                    headers_squad_standard.append("Année édition")
                    stats_schedules_data["matchs"].append(headers_squad_standard)


                body_table_standard_stat_by_squad = table_standard_stat_by_squad.find("tbody")
                tr_headers_standard_by_squad = body_table_standard_stat_by_squad.find_all("tr")
                for tr in tr_headers_standard_by_squad:
                    row = []
                    th_round = tr.find("th")
                    if th_round != None:
                        th_a_round = th_round.find("a")
                        if th_a_round != None:
                            row.append(th_a_round.text)
                    for td in tr.find_all("td"):
                        td_a = td.find("a")
                        if td_a != None:
                            row.append(td_a.text)
                        else :
                            row.append(td.text)
                    row.append(annee_edition)
                    stats_schedules_data["matchs"].append(row)


    return stats_schedules_data    

all_schedules_stats = get_schedules_stat("schedule", urls, "sched_all")

#for url in urls["schedule"]:
    #checking_url = check_access_url(requests.get(url))
    #if checking_url != True:
    #    print(url)

prends une pause et reviens dans 3600 secondes


In [377]:
schedule_stats = list()
for kk in range(len(all_schedules_stats["matchs"])):
    count = 0
    for i in all_schedules_stats["matchs"][kk]:
        if "" == i:
            count+=1
    if count <8 :
        print(all_schedules_stats["matchs"][kk])
        schedule_stats.append(all_schedules_stats["matchs"][kk])
file_path_schedule_stat = "datas_project/schedule_stats.csv"
save_data_frame_to_csv(schedule_stats, file_path_schedule_stat)

['Round', 'Wk', 'Day', 'Date', 'Time', 'Home', 'xG', 'Score', 'xG', 'Away', 'Attendance', 'Venue', 'Referee', 'Match Report', 'Notes', 'Année édition']
['Group stage', '1', 'Tue', '2022-09-06', '18:45 ', 'Dinamo Zagreb', '0.6', '1–0', '0.9', 'Chelsea', '20,607', 'Stadion Maksimir', 'István Kovács', 'Match Report', '', '2022-2023']
['Group stage', '1', 'Tue', '2022-09-06', '18:45 ', 'Dortmund', '2.3', '3–0', '0.5', 'FC Copenhagen', '70,700', 'Signal Iduna Park', 'François Letexier', 'Match Report', '', '2022-2023']
['Group stage', '1', 'Tue', '2022-09-06', '20:00 ', 'Celtic', '1.4', '0–3', '1.9', 'Real Madrid', '57,057', 'Celtic Park', 'Sandro Schärer', 'Match Report', '', '2022-2023']
['Group stage', '1', 'Tue', '2022-09-06', '20:00 ', 'Benfica', '1.1', '2–0', '0.2', 'Maccabi Haifa', '55,130', 'Estádio do Sport Lisboa e Benfica', 'Andreas Ekberg', 'Match Report', '', '2022-2023']
['Group stage', '1', 'Tue', '2022-09-06', '21:00 ', 'RB Salzburg', '1.3', '1–1', '0.9', 'Milan', '29,520', 